# Construct cell from two equivalent atoms 

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.base import Int
from aiida.work.workfunction import workfunction

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview

from structure_browser import StructureBrowser

## Step 1: Select a structure from the AiiDA database

In [ ]:
def on_struct_change(c):
    global selection, viewer, cell_ready

    # remove old components
    if hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        #viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    # empty selection
    selection = set()
    cell_ready = False

    node = struct_browser.results.value
    if node:
        # add new component
        atoms = node.get_ase()
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        #viewer.add_unitcell()
        viewer.center_view()

In [ ]:
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')
display(struct_browser)

## Step 2: Select two equivalent atoms to guide the unit cell construction

In [ ]:
def on_picked(c):
    global selection, cell_ready
    
    cell_ready = False
    
    if 'atom' not in viewer.picked.keys():
        return # did not click on atom
    with picked_out:
        clear_output()
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.add_ball_and_stick()
        #viewer.add_unitcell()

        idx = viewer.picked['atom']['index']

        # toggle
        if idx in selection:
            selection.remove(idx)
        else:
            selection.add(idx)

        #if(selection):
        sel_str = ",".join([str(i) for i in sorted(selection)])
        print("Selected atoms: "+ sel_str)
        viewer.add_representation('ball+stick', selection="@"+sel_str, color='red', aspectRatio=3.0)
        #else:
        #    print ("nothing selected")
        viewer.picked = {} # reset, otherwise immidiately selecting same atom again won't create change event

selection = set()    
viewer = nglview.NGLWidget()
viewer.observe(on_picked, names='picked')
picked_out = ipw.Output()
display(viewer, picked_out)

## Step 3: Construct unit cell and check results

In [ ]:
def on_click_cell(b):
    global cell_ready
    
    with clickcell_out:
        clear_output()
        cell_ready = False
        inp_descr.value = ""
        
        if hasattr(viewer2, "component_0"):
            viewer2.component_0.remove_ball_and_stick()
            viewer2.component_0.remove_unitcell()
            cid = viewer2.component_0.id
            viewer2.remove_component(cid)   

        if len(selection) != 2:
            print("You must select exactly two atoms")
            return 

        orig_structure = struct_browser.results.value
        id1 = sorted(selection)[0]
        id2 = sorted(selection)[1]
        new_structure = construct_cell(orig_structure, Int(id1), Int(id2))
        formula = new_structure.get_ase().get_chemical_formula()
           
        #inp_descr.value = orig_structure.description + ".construct(%i, %i)"%(id1, id2) + formula
        inp_descr.value = "constructe("+formula+")"
        cell_ready = True
        
         # search for existing structures
        qb = QueryBuilder()
        qb.append(StructureData, filters={'extras.formula': formula})
        qb.order_by({StructureData:{'ctime':'desc'}})
        for n in qb.iterall():
            print("Found existing structure: PK=%d | %s"%(n[0].pk, n[0].description))
    
        # display in second viewer
        atoms2 = new_structure.get_ase()
        viewer2.add_component(nglview.ASEStructure(atoms2)) # adds ball+stick
        #viewer2.add_ball_and_stick()
        viewer2.add_unitcell()
        viewer2.center_view()

btn_cell = ipw.Button(description='Construct Cell')
btn_cell.on_click(on_click_cell)
clickcell_out = ipw.Output()
display(btn_cell, clickcell_out)

In [ ]:
viewer2 = nglview.NGLWidget()
display(viewer2)

In [ ]:
def construct_cell(orig_struct, id1, id2):
    id1, id2 = id1.value, id2.value
    atoms = orig_struct.get_ase()
    
    p1 = [atoms[id1].x, atoms[id1].y]
    p0 = [atoms[id2].x, atoms[id2].y]
    p2 = [atoms[id2].x, atoms[id1].y]
    
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))

    #angle=np.degrees(angle)

    yrange = np.amax(atoms.positions[:,1])-np.amin(atoms.positions[:,1])
    zrange = np.amax(atoms.positions[:,2])-np.amin(atoms.positions[:,2])
    
    cx = norm(v0)
    cy = 15.0 + yrange
    cz = 15.0 + zrange
    #print np.degrees(angle),v0,v1,p0,p1
    if np.abs(angle) > 0.01:
    #   s.euler_rotate(phi=angle,theta=0,psi=0,center(x[id1],y[id1],z[id1]))
        atoms.rotate_euler(center=atoms[id1].position, phi=-angle,theta=0.0,psi=0.0)
    atoms.cell = (cx,cy,cz)
    atoms.pbc = (True,True,True)
    atoms.wrap(eps=0.001)
    atoms.center()

    #### REMOVE REDUNDANT ATOMS
    tobedel = []

    cov_radii = [covalent_radii[a.number] for a in atoms]
    nl = NeighborList(cov_radii, bothways = False, self_interaction = False)
    nl.update(atoms)

    for a in atoms:
        indices, offsets = nl.get_neighbors(a.index)
        for i, offset in zip(indices, offsets):
            dist = norm(a.position -(atoms.positions[i] + np.dot(offset, atoms.get_cell())))
            if dist < 0.4 :
                tobedel.append(atoms[i].index)

    del atoms[tobedel]
    #s.write("unit.xyz")
    #### END REMOVE REDUNDANT ATOMS

    #s.cell=[cx,cy,cz]
    #s.set_cell(s.cell/(1.54/1.41333),scale_atoms=True)
    #s.cell=[cx/scale_armchair,cy,cz]
    #s.center()



    #### ENDFIND UNIT CELL AND APPLIES IT


    #### ADD Hydrogens
    cov_radii = [covalent_radii[a.number] for a in atoms]
    nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
    nl.update(atoms)

    need_a_H = []
    for a in atoms:
        nlist=nl.get_neighbors(a.index)[0]
        if len(nlist)<3:
            if a.symbol=='C':
                need_a_H.append(a.index)

    print "Added missing Hydrogen atoms: ", need_a_H

    dCH=1.1
    for a in need_a_H:
        vec = np.zeros(3)
        indices, offsets = nl.get_neighbors(atoms[a].index)
        for i, offset in zip(indices, offsets):
            vec += -atoms[a].position +(atoms.positions[i] + np.dot(offset, atoms.get_cell()))
        vec = -vec/norm(vec)*dCH
        vec += atoms[a].position
        htoadd = ase.Atom('H',vec)
        atoms.append(htoadd)

    return StructureData(ase=atoms)

## Step 4: Store structure in the AiiDA database

In [ ]:
def on_click_store(b):
    if not cell_ready:
        print("Unit cell has to be constructed first.")
        return

    # construct cell again, this time using a workfunction
    orig_structure = struct_browser.results.value
    id1 = Int(sorted(selection)[0])
    id2 = Int(sorted(selection)[1])
    s = construct_cell_wf(orig_structure, id1, id2)
    s.description = inp_descr.value
    s.store()
    print("Stored in AiiDA: "+repr(s))

cell_ready = False
inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
display(ipw.HBox([btn_store, inp_descr]))

In [ ]:
@workfunction
def construct_cell_wf(orig_struct, id1, id2):
    return construct_cell(orig_struct, id1, id2)